# Equal-Weight S&P 500 Index Fund.

In [1]:
# Import libraries

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
# Import list of stocks

stocks = pd.read_csv("sp_500_stocks.csv")
stocks = stocks.drop([135,219, 467, 484]) # stock giving error, might not be in list

In [3]:
# Import IEX Cloud API Token (Sandbox API)

from secret import IEX_CLOUD_API_TOKEN
token = IEX_CLOUD_API_TOKEN

In [5]:
# Make first API call

symbol = "CME"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={token}"
data = requests.get(api_url).json()
# data

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
final_df = pd.DataFrame(columns = my_columns)

for stock in stocks["Ticker"]:
    
    # Make API Call for each stock
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={token}"
    response = requests.get(api_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
         

    # Parse API Call
    price = data['latestPrice']
    market_cap = data['marketCap']
    
    # Add stocks to dataframe
    new_stock = pd.DataFrame([[stock, price, market_cap, "N/A"]], columns = my_columns)
    final_df = pd.concat([final_df, new_stock])

In [7]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,150.320,44471357362,N/A
0,AAL,12.960,8472189842,N/A
0,AAP,155.200,9390785931,N/A
0,AAPL,129.610,2069207580733,N/A
0,ABBV,163.527,295197245044,N/A
...,...,...,...,...
0,YUM,127.396,37265852424,N/A
0,ZBH,131.350,27822279469,N/A
0,ZBRA,266.270,13606235795,N/A
0,ZION,50.830,7549483151,N/A


## Using Batch API Calls to improve performance.

In [8]:
def divide_chunks(lst, n):     
    """Yield successive n sized chunks/batches from lst"""
    for i in range(0, len(lst), n):
        yield lst.loc[i:i + n]

In [9]:
symbol_groups = list(divide_chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    # Make Batch API Call 
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={token}"
    response = requests.get(batch_api_call_url)
    response.raise_for_status()  # raises exception when not a 2xx response
    if response.status_code != 204:
        data = response.json()
    
    for symbol in symbol_string.split(','):
        
        if symbol in data.keys():
            # Parse API Call
            price = data[symbol]['quote']['latestPrice']
            market_cap = data[symbol]['quote']['marketCap']

            # Add stocks to dataframe
            new_stock = pd.DataFrame([[symbol, price, market_cap, "N/A"]], columns = my_columns)
            final_df = pd.concat([final_df, new_stock])

In [10]:
final_df = final_df.set_index("Ticker").reset_index()
final_df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,150.250,45294540030,N/A
1,AAL,13.392,8431300312,N/A
2,AAP,151.280,9050847964,N/A
3,AAPL,127.650,1999966491661,N/A
4,ABBV,163.730,288409324400,N/A
...,...,...,...,...
499,YUM,133.084,35952306667,N/A
500,ZBH,128.920,27350811151,N/A
501,ZBRA,263.260,14014681409,N/A
502,ZION,50.560,7427001616,N/A


## Calculating the number of shares to buy.

In [11]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("Please enter an integer.")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:1000000
1000000.0


In [23]:
position_size = val/len(final_df.index)
final_df['Number of shares to buy'] = (position_size/final_df['Stock Price']).apply(np.floor)
final_df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,150.250,45294540030,13.0
1,AAL,13.392,8431300312,148.0
2,AAP,151.280,9050847964,13.0
3,AAPL,127.650,1999966491661,15.0
4,ABBV,163.730,288409324400,12.0
...,...,...,...,...
499,YUM,133.084,35952306667,14.0
500,ZBH,128.920,27350811151,15.0
501,ZBRA,263.260,14014681409,7.0
502,ZION,50.560,7427001616,39.0


## Formatting Excel Output

In [39]:
# Initialize xlsxwriter object
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index = False)


In [40]:
# Set formats
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

In [41]:
# Apply formats
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format], 
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of shares to buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f"{column}:{column}", 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f"{column}1", column_formats[column][0], column_formats[column][1])

In [42]:
# Save .xlsx file
writer.save()

<ipython-input-42-16d93d5523a4>:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
